# Extract phenology from OzFlux GPP

This is to show that the algorithm performs well regardless of NDVI or GPP

In [ ]:
%matplotlib inline
import os
import sys
import pingouin as pg
import xarray as xr
import numpy as np
import pandas as pd
import geopandas as gpd
import scipy.signal
import contextily as ctx
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sb
from odc.geo.xr import assign_crs
import matplotlib.colors as colors
import distinctipy
from matplotlib.colors import LinearSegmentedColormap
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import warnings
warnings.simplefilter(action='ignore')

import sys
sys.path.append('/g/data/os22/chad_tmp/Aus_phenology/src')
from phenology_pixel_circular import xr_phenometrics

In [ ]:
# # Savitsky-Golay smoothing function
def sg_smooth(ds, window, poly, deriv):
    return xr.apply_ufunc(
        scipy.signal.savgol_filter,
        ds,
        input_core_dims=[['time']],
        output_core_dims=[['time']],
        kwargs=dict(
            window_length=window,
            polyorder=poly,
            deriv=deriv,
            mode='interp'),
        dask='parallelized'
    )

def filter_complete_years(ds):
    # Extract year and month information
    df = ds.time.to_dataframe().reset_index(drop=True)
    df['year'] = df['time'].dt.year
    df['month'] = df['time'].dt.month
    
    # Identify years with all 12 months
    complete_years = (
        df.groupby('year')['month']
        .nunique()
        .loc[lambda x: x == 12]  # Keep only years with exactly 12 unique months
        .index
    )
    
    # Filter the data array to include only these complete years
    filtered_data = ds.where(ds.time.dt.year.isin(complete_years), drop=True)
    
    return filtered_data

## Remote sensing data

In [ ]:
ds = assign_crs(xr.open_dataset('/g/data/os22/chad_tmp/AusENDVI/results/publication/AusENDVI-clim_MCD43A4_gapfilled_1982_2022_0.2.0.nc'), crs='EPSG:4326')
ds = ds.rename({'AusENDVI_clim_MCD43A4':'NDVI'})
ds = ds['NDVI']

rain = xr.open_dataset('/g/data/os22/chad_tmp/Aus_phenology/data/rainfall_ANUClim_daily_5km_1960_2022.nc')['rain']
rain = assign_crs(rain.sel(time=slice('1982', '2022')), crs='EPSG:4326')

ds_8day = xr.open_dataset('/g/data/os22/chad_tmp/Aus_phenology/data/NDVI_mcd43a4_8day_2014_2020.nc')['NDVI']
ds_8day = assign_crs(ds_8day, 'epsg:4326')

## Flux tower data

#### Process timeseries and calculate phenometrics

In [ ]:
flux = xr.open_dataset('https://dap.tern.org.au/thredds/dodsC/ecosystem_process/ozflux/AliceSpringsMulga/2022_v2/L6/default/AliceSpringsMulga_L6_Daily.nc')['GPP_SOLO']

flux  = xr.where(flux<0, 0, flux)

# Index NDVI at location and time so we have matching time series
lat,lon = flux.latitude, flux.longitude

# flux = filter_complete_years(flux)
flux=sg_smooth(flux, window=60, poly=3, deriv=0)

flux_phen = xr_phenometrics(flux, prominence=0.1, soil_signal=0.0).compute()

## Process NDVI data

In [ ]:
ds_8day = ds_8day.sel(latitude=lat, longitude=lon, method='nearest')
ndvi_monthly = ds.sel(latitude=lat, longitude=lon, method='nearest')
rain = rain.sel(latitude=lat, longitude=lon, method='nearest')

ndvi = ndvi_monthly.resample(time="2W").interpolate("linear")
ndvi=sg_smooth(ndvi,  window=11, poly=3, deriv=0)
ndvi = ndvi.dropna(dim='time',
        how='all').resample(time='1D').interpolate(kind='quadratic')

# # #phenology
ndvi_phen = xr_phenometrics(ndvi, prominence=0.005, soil_signal=0.15).compute()



In [ ]:
# ts_daily = flux_tss[k].sel(time=slice('2012','2020'))
# ts_monthly = flux_tss_orig[k].sel(time=slice('2012','2020'))
# flux_pheno = flux_pheno[k].sel(index=range(0,10))

In [ ]:
fig,ax=plt.subplots(1,1, figsize=(10,4))

ax2 = ax.twinx()

flux.sel(time=slice('2011','2016')).plot(ax=ax, c='tab:blue', linestyle='--', linewidth=1.0, label='Daily GPP')

ndvi_monthly.sel(time=slice('2011','2016')).plot(ax=ax2, c='tab:red', linestyle='--', linewidth=1.0, label='Monthly NDVI')

# ndvi.sel(time=slice('2013','2014')).squeeze().drop_vars(['latitude', 'longitude']).plot(ax=ax2, label='NDVI daily')

ax.legend(loc='upper left')
ax2.legend(loc='upper right')
ax.set_title('Alice Springs Mulga flux tower site')
ax2.set_title(None)
ax.grid(alpha=0.75)
ax.set_ylabel('GPP gC/m\N{SUPERSCRIPT TWO}/day')
ax2.set_ylabel('NDVI')
ax2.set_xlabel(None)
ax.set_xlabel(None);
